In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader
cudnn.benchmark = True
plt.ion()   # interactive mode
from utils.ImagesDataset import ImagesDataset
from tqdm import tqdm
CLASSES=14

# Dataset

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
        transforms.RandomRotation(20),
        transforms.RandomAdjustSharpness(1.5),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])]),
}
train_set = ImagesDataset(data_transforms['train'],noval=True)
val_set = ImagesDataset(data_transforms['val'],train=False)
val_set.__len__()

66829

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu")

### dataloader

In [5]:
bs=128
dataset_sizes = {'train':train_set.__len__(),'val': val_set.__len__()}
train_loader = DataLoader(dataset=train_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4)
val_loader = DataLoader(dataset=val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4)
dataloaders={'train':train_loader,'val':val_loader}

# Training

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    hist=[]
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
   
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            with tqdm(dataloaders[phase], unit="batch") as tepoch:
                batches=0
                for inputs, labels in tepoch:
                    tepoch.set_description(f"Epoch {epoch}")
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()
                    
                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        #print('out',outputs)
                        #print('labels',labels)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    batches+=1                      
                    tepoch.set_postfix(loss=running_loss/(batches*bs), accuracy=100. * running_corrects.item()/(batches*bs))
                    hist.append([running_loss/(batches*bs),100. * running_corrects.item()/(batches*bs)])
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model,hist

### setup

In [7]:
model_ft = models.efficientnet_b0(pretrained=True)

model_ft.classifier[1] = nn.Linear(in_features=1280, out_features=CLASSES)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9,weight_decay=1e-4)

# Decay LR by a factor of 0.1 every 2 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.1)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /home/ecbm4040/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [8]:
model_ft,hist = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=4)

Epoch 0/3
----------


Epoch 0: 100%|████████████████████████████████████| 1681/1681 [13:00<00:00,  2.15batch/s, accuracy=76.7, loss=0.73]


train Loss: 0.7300 Acc: 0.7678


Epoch 0: 100%|█████████████████████████████████████| 523/523 [02:04<00:00,  4.21batch/s, accuracy=84.8, loss=0.432]


val Loss: 0.4324 Acc: 0.8495

Epoch 1/3
----------


...

val Loss: 0.2860 Acc: 0.9012

Epoch 3/3
----------


Epoch 3: 100%|█████████████████████████████████████| 1681/1681 [12:00<00:00,  2.33batch/s, accuracy=89, loss=0.321]


train Loss: 0.3215 Acc: 0.8904


Epoch 3: 100%|█████████████████████████████████████| 523/523 [02:00<00:00,  4.33batch/s, accuracy=90.3, loss=0.279]


val Loss: 0.2791 Acc: 0.9041

Training complete in 57m 13s
Best val Acc: 0.904069


In [10]:
torch.save(model_ft.state_dict(), './models/efficientnet_b0_noval.pt')
import pickle
with open('./models/efficientnet_b0_hist.pkl', 'wb') as f:
    pickle.dump(hist, f)

# Evaluation

In [20]:
model_ft = models.efficientnet_b0(pretrained=True)

model_ft.classifier[1] = nn.Linear(in_features=1280, out_features=CLASSES)
model_ft = model_ft.to(device)


#reloading saved weights
model_ft.load_state_dict(torch.load('./models/efficientnet_b0.pt'))
model_ft.eval()



#evaluation
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    with tqdm(dataloaders['val'], unit="batch") as tepoch:
        for images, labels in tepoch:
            tepoch.set_description('progress lolol')
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_ft(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Validation accuracy : {acc} %')

progress lolol: 100%|██████████████████████████████████████████████████████| 523/523 [02:08<00:00,  4.07batch/s]

Validation accuracy : 78.8445136093612 %


In [21]:
val_set = ImagesDataset(train=False)

#resnet model
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, CLASSES)
model_ft = model_ft.to(device)


#reloading saved weights
model_ft.load_state_dict(torch.load('./models/resnet18.pt'))
model_ft.eval()



#evaluation
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    with tqdm(dataloaders['val'], unit="batch") as tepoch:
        for images, labels in tepoch:
            tepoch.set_description('progress lolol')
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_ft(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Validation accuracy : {acc} %')

progress lolol: 100%|██████████████████████████████████████████████████████| 523/523 [02:02<00:00,  4.26batch/s]

Validation accuracy : 78.3417378682907 %


#Let there be 9 samples and 1 sample in class 0 and 1 respectively
class_counts = [9.0, 1.0]
num_samples = sum(class_counts)
labels = [0, 0,..., 0, 1] #corresponding labels of samples

class_weights = [num_samples/class_counts[i] for i in range(len(class_counts))]
weights = [class_weights[labels[i]] for i in range(int(num_samples))]
sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))